# This Scala Program is used to model the pipe line for the Prediction

### 1.0 Importing Libaries 

In [1]:
import  org.apache.spark._ 
import  spark.implicits._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Imputer
import org.apache.spark.sql._
import SQLContext._
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
import scala.util.hashing.MurmurHash3
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator  
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics  
import org.apache.spark.ml.classification.LogisticRegression  
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}  
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator,VectorIndexer}  
import org.apache.spark.ml.linalg.Vectors  
import org.apache.spark.ml.{Pipeline,PipelineModel}  
import org.apache.log4j._  
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{IndexToString, StringIndexer}
import org.apache.spark.sql.DataFrame
import java.io.{File, PrintWriter}
Logger.getLogger("org").setLevel(Level.ERROR) 

Intitializing Scala interpreter ...

Spark Web UI available at http://f34dae46e575:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591625542639)
SparkSession available as 'spark'


import org.apache.spark._
import spark.implicits._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Imputer
import org.apache.spark.sql._
import SQLContext._
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
import scala.util.hashing.MurmurHash3
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache....

## 2.0 Starting spark session

In [2]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@58a19422


# 3.0 Importing the Cleaned file - Output file from the Wrangling Program

In [3]:
val data = spark.read.option("header","true").
    option("inferSchema","true").
    format("csv").
    load("/home/work/weatherAUS-clean-FINAL.csv")

data: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 31 more fields]


In [4]:
data.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- RainToday_Sc: integer (nullable = true)
 |--

#  4.0 Selecting required fields with right data type

### 4.1 Feature Selection

<b>  Based on  Feature selection done using the correlation matrix (separate Scala program attached ) following filelds were identified to be the most  influencing field for   the rainfall on the following day </b>

In [5]:
val df1 = data.selectExpr("Date","Location","MaxTemp","Humidity3pm", "Pressure3pm","Temp3pm","Temp9am","RainToday_Sc",
                         "cast (Month as String) Month","RainTomorrow_Sc")
                        

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [6]:
df1.count()

res2: Long = 142193


### 4.2 Filtering and forming a particular location data

<b>Australia having very distinct 8 climate zones,no single model will work for all australia  Even within a state  there are multiple climate zones(from alpine to high humid) 
Different models need to be trained for different locations
In this approach,separate models will be created for different locations and saved in the directory for the prediction particular location model will be loaded and prediction will be done
Data used was weatherAUS_YN.csv(before imputing the  null values) as it was  observed  that 
dropping null values are more effective than imputing (In our scenario imputing means blunting the features i.e.  generalising the data)
Wind direction (categorical columns) not tested or used in the modelling as too much of encoding required and crashing the docker
correlation matrix(wrangling file)& trial & Error methods were used to select the features) </b>

### 4.3  Dropping the null values if any before the modeling 

In [7]:
val Loc =df1.na.drop()

Loc: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [8]:
Loc.count()

res3: Long = 130412


<b> We have decided to impute NA with Nearest values ( previous day or next day) or monthly average. Even then for some of the weather stations the data for the whole month is empty . Hence we were not able to impute with suitable values.

Hence our group decided that  we will remove NA rows  as it might impact drastically the model efficiency </b>

## 5.0 Modeling  - Data Preparation

<b>The problem is to predict the rain for tomorrow based on Labeled data set . Hence it is Supervised Learning probelm. Moreover since it is to predict whether to rain or not , it is Binary classification. We have decided to model both logistic regression and Random forrest and adopt a model which is best suited for the dataset </b>

### 5.1 Setting the label column(Rain tomorrow)

In [9]:
val df2 = (Loc.select(Loc("RainTomorrow_Sc").as("label"), 
           $"Location",
           $"Date",
          $"MaxTemp", 
         $"Month",
         $"Raintoday_Sc",
          $"Humidity3pm",
          $"Pressure3pm", 
           $"Temp9am", 
          $"Temp3pm"))

df2: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 8 more fields]


### 5.2 Indexing the month column(categorical)-To capture the seasonal pattern

In [10]:
// Month  which is a categorical variable need  to be indexed and encoded to be used in logistic regression

In [11]:
val Indexer=new StringIndexer().setInputCol("Month").setOutputCol("MonIn")


Indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_f84e9bcd3585


### 5.3  Encoding the MonIn( column)

In [12]:
val Encoder=new OneHotEncoderEstimator().setInputCols(Array("MonIn")).setOutputCols(Array("MonEn"))


Encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_1500fad5a17b


### 5.4  Setting the assembler

In [13]:
//setting the assembler with required fields for the regression model

In [14]:
 val assembler=(new VectorAssembler().setInputCols(Array(
                                                        "Raintoday_Sc",
                                                         "MaxTemp",
                                                        "Humidity3pm",
                                                         "Temp3pm",
                                                         "MonEn",
                                                        "Pressure3pm"
                                                        )).setOutputCol("features"))

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_7d010328c20c


### 5.5 Splitting the data into test and train sets

In [15]:
val Array(training,test)=df2.randomSplit(Array(0.7,0.3),seed=999)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]


## 6.0  Logistic regression

In [16]:
//Maximum iteration was set to 20,more iterations giving minor improvements in the accuracy but more runtime and docker failures
// as a trade off maximum iteration was set to 10

### 6.1 Create an instance of the Logistic Regression

In [17]:
val lr=new LogisticRegression().setMaxIter(20)
       

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_56df69446b3e


### 6.2  Setting the pipeline

In [18]:
val pipeline=new Pipeline().setStages(Array(Indexer,Encoder,assembler,lr))

pipeline: org.apache.spark.ml.Pipeline = pipeline_afbf47bb8b08


### 6.3  Cross validation hyper parameter tuning

In [19]:
val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0.1, 0.01))
  .build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_56df69446b3e-regParam: 0.1
}, {
	logreg_56df69446b3e-regParam: 0.01
})


In [20]:
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new BinaryClassificationEvaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3)  

cv: org.apache.spark.ml.tuning.CrossValidator = cv_57b97ffed324


In [ ]:
val cvModel = cv.fit(training)

2020-06-08 14:13:13,110 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2020-06-08 14:13:13,116 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


### 6.4 Transforming test data with model

In [ ]:
val results =cvModel.transform(test)

### 6.5  Model Evaluation

In [ ]:
val predictions=results.select($"prediction",$"label").as[(Double,Double)].rdd

In [ ]:
val metrics= new MulticlassMetrics(predictions)

In [ ]:
println(metrics.confusionMatrix)

(1,1) no  pred no  rain
(2,2) yes pred yes rain
(1,2) no pred yes rain 
(2,1)  yes pred no rain 

In [ ]:
metrics.accuracy

In [ ]:
Testing

### 6.6 Saving the model

In [ ]:
cvModel.write.overwrite().save("/home/model_Sydney")

### 6.7 Saving the pipeline

In [ ]:
pipeline.write.overwrite().save("/home/pipeline_Sydney")

### 6.8  Reloading the model

In [ ]:
val modelLoaded=CrossValidatorModel.load("/home/model_Sydney")

### 6.9 Using the model to run on Test Prediction using Test date

In [ ]:
// loaded model is applied on dataframe to see the prediction vs label

In [ ]:
val dfu= modelLoaded.transform(df2)

In [ ]:
df2.show() //Trying to check the actual Vs prediction for the sydney for last 10 years

In [ ]:
val testev=dfu.select("Location","Date","RawPrediction","probability","label","prediction").toDF()

In [ ]:
testev.show()

In [ ]:
// future ideas to expand
// creating models to different climate zones
// trying to predict next couple of days rainfall( e.g. 3 days) with the available data
// Trying deep learning neural networks for better models
// Estimate the amount of rainfall

# 7.0 Random Forrest Modeling 

<b>Data set prepared in step 5   will be reused for Random Forrest as well </b>


### 7.1 Create Instance of Random Forrest

In [ ]:
val rf = new RandomForestClassifier()
      .setLabelCol("label")
      .setFeaturesCol("features")
      .setNumTrees(10)
      .setMaxBins(41)
rf

### 7.2 Setting the pipeline

In [ ]:
val M_pipeline=new Pipeline().setStages(Array(Indexer,Encoder,assembler,rf))

### 7.3 Training the Model

In [ ]:
val Trg_model=M_pipeline.fit(training)

### 7.4 Save the Training Model

In [ ]:
Trg_model.write
  .overwrite()
  .save("spark-RandomForrest-model")

### 7.5 Load the model to try with test data for prediction

In [ ]:
val RegModel = PipelineModel.load("spark-RandomForrest-model")

### 7.6 Run Prediction with the test data 

In [ ]:
val results =RegModel.transform(test)

### 7.7 Evaluate the Results

In [ ]:
val evaluator = new MulticlassClassificationEvaluator()
      .setLabelCol("label")
      .setPredictionCol("prediction")
      .setMetricName("accuracy")

 <b>Check the Accuracy of the Model</b>

In [ ]:
val accuracy = evaluator.evaluate(results)